In [36]:
import os 

In [37]:
%pwd

'd:\\exercises\\Cold_Email_Generator\\MLOPS\\End-to-End-Machine-Learning-Project-with-MLFLOW'

In [38]:
# os.chdir('../')

In [39]:
%pwd

'd:\\exercises\\Cold_Email_Generator\\MLOPS\\End-to-End-Machine-Learning-Project-with-MLFLOW'

In [40]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path : Path
    test_path : Path
    metrics_path : Path
    all_params : dict
    target_column : str
    mlflow_uri : str

In [41]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories, save_json
from ML_Project import logger
from dataclasses import dataclass
from pathlib import Path



class ConfigurationManager:
    def __init__(self,
                 config_path = Config_File_Path,
                 params_path = Params_File_Path,
                 schema_path = Schema_File_Path):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        # print(self.schema)
        create_directories([self.config['artifacts_root']])

    def get_model_eval_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            test_path = config.test_path,
            metrics_path = config.metrics_path,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = 'https://dagshub.com/dipan97-hue/End-to-End-Machine-Learning-Project-with-MLFLOW.mlflow'
            
        )
        return model_evaluation_config


        

In [42]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from ML_Project import logger

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test = pd.read_csv(self.config.test_path)
        model = joblib.load(self.config.model_path)

        test_x = test.drop([self.config.target_column], axis =1)
        test_y = test[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }

            save_json(path = Path(self.config.metrics_path), data = scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_uri != "file":
                mlflow.sklearn.log_model(model, "model" , registered_model_name="ElasticNetWineModel")
            else:
                mlflow.sklearn.log_model(model, "model")

            logger.info(f"Model logged into MLFlow")
            






In [43]:
try:
    config_manager = ConfigurationManager()
    model_eval_config = config_manager.get_model_eval_config()
    model_evaluation = ModelEvaluation(config=model_eval_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    logger.error(e)
    raise e

[2024-12-30 21:46:24,098: INFO: common : Yaml file loadded successfully from config\config.yaml]
[2024-12-30 21:46:24,138: INFO: common : Yaml file loadded successfully from params.yaml]
[2024-12-30 21:46:24,146: INFO: common : Yaml file loadded successfully from schema.yaml]
[2024-12-30 21:46:24,156: INFO: common : Directory created at artifacts]
[2024-12-30 21:46:24,159: INFO: common : Directory created at artifacts/model_evaluation]


[2024-12-30 21:46:24,434: INFO: common : Data saved at artifacts\model_evaluation\metrics.json]


2024/12/30 21:46:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2024-12-30 21:46:31,886: INFO: 3728083176 : Model logged into MLFlow]
